In [119]:
import requests 
import json
import pandas as pd
import numpy as np
from scipy import stats
from sodapy import Socrata
import gmaps
from pandas.io.json import json_normalize


In [120]:
client = Socrata("chronicdata.cdc.gov", None)
url = "https://chronicdata.cdc.gov/resource/rb39-ixr6.json"
results = client.get("rb39-ixr6", limit=38708)

In [121]:
#results

In [122]:
#print(requests.get(url))

In [123]:
#print(requests.get(url).json())

In [124]:
#response = requests.get(url).json()
#print(json.dumps(response, indent = 4, sort_keys=True))
#marked out due to API stuff discussed with Brent

In [125]:
health_df = pd.DataFrame(results)
health_df.count()

:@computed_region_bxsw_vy29    37220
:@computed_region_he4y_prf8    37750
age                            38708
data_value                     24275
data_value_footnote            14433
data_value_footnote_symbol     14433
data_value_std_err             24275
data_value_type                38708
data_value_unit                38708
datasource                     38708
displayorder                   38708
education                      38708
gender                         38708
geolocation                    38708
high_confidence_limit          24275
locationabbr                   38708
locationdesc                   38708
low_confidence_limit           24275
measuredesc                    38708
measureid                      38708
race                           38708
response                       38708
sample_size                    24275
stratificationid1              38708
stratificationid2              38708
stratificationid3              38708
stratificationid4              38708
s

In [126]:
#Collecting a list of all columns within the DataFrame/
health_df.columns

Index([':@computed_region_bxsw_vy29', ':@computed_region_he4y_prf8', 'age',
       'data_value', 'data_value_footnote', 'data_value_footnote_symbol',
       'data_value_std_err', 'data_value_type', 'data_value_unit',
       'datasource', 'displayorder', 'education', 'gender', 'geolocation',
       'high_confidence_limit', 'locationabbr', 'locationdesc',
       'low_confidence_limit', 'measuredesc', 'measureid', 'race', 'response',
       'sample_size', 'stratificationid1', 'stratificationid2',
       'stratificationid3', 'stratificationid4', 'submeasure', 'submeasureid',
       'topicdesc', 'topicid', 'topictype', 'topictypeid', 'year'],
      dtype='object')

In [148]:
# Reorganizing the columns using double brackets
organized_df = health_df[["year","locationdesc","topicdesc","measuredesc","submeasure","data_value","sample_size","gender","age","race","education", "geolocation"]]
organized_df.head()

,year,locationdesc,topicdesc,measuredesc,submeasure,data_value,sample_size,gender,age,race,education,geolocation
0,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School,"{'type': 'Point', 'coordinates': [-106.2405809..."
1,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School,"{'type': 'Point', 'coordinates': [-97.52107021..."
2,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School,"{'type': 'Point', 'coordinates': [-75.54397042..."
3,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School,"{'type': 'Point', 'coordinates': [-86.63186076..."
4,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-81.04537120..."


In [149]:
latlong2 = (pd.concat({i: json_normalize(x) for i, x in organized_df.pop('geolocation').items()})
         .reset_index(level=1, drop=True)
         .join(organized_df)
         .reset_index(drop=True))
latlong2.head()

,coordinates,type,year,locationdesc,topicdesc,measuredesc,submeasure,data_value,sample_size,gender,age,race,education
0,"[-106.240580985, 34.520880952]",Point,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School
1,"[-97.521070214, 35.472031356]",Point,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School
2,"[-75.543970427, 42.827001032]",Point,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School
3,"[-86.631860762, 32.840571122]",Point,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School
4,"[-81.045371207, 33.998821303]",Point,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School


In [158]:
latlong3 = pd.DataFrame(latlong2["coordinates"].values.tolist())
latlong3.head()

,0,1
0,-106.240581,34.520881
1,-97.521070,35.472031
2,-75.543970,42.827001
3,-86.631861,32.840571
4,-81.045371,33.998821


In [160]:
renamedlatlong = latlong3.rename(columns={0:"Lat",1:"Lon"})
renamedlatlong.head()

,Lat,Lon
0,-106.240581,34.520881
1,-97.521070,35.472031
2,-75.543970,42.827001
3,-86.631861,32.840571
4,-81.045371,33.998821


In [170]:
new_df = pd.merge(organized_df, renamedlatlong, left_index=True, right_index=True, how='inner')
new_df.head()

,year,locationdesc,topicdesc,measuredesc,submeasure,data_value,sample_size,gender,age,race,education,Lat,Lon
0,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School,-106.240581,34.520881
1,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School,-97.521070,35.472031
2,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School,-75.543970,42.827001
3,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School,-86.631861,32.840571
4,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School,-81.045371,33.998821


In [171]:
organized_df = new_df
organized_df.head()

,year,locationdesc,topicdesc,measuredesc,submeasure,data_value,sample_size,gender,age,race,education,Lat,Lon
0,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School,-106.240581,34.520881
1,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School,-97.521070,35.472031
2,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School,-75.543970,42.827001
3,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School,-86.631861,32.840571
4,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School,-81.045371,33.998821


In [172]:
renamed_df = organized_df.rename(columns={"year":"Year","locationdesc":"State","topicdesc":"Use Type","measuredesc":"Status","submeasure":"Frequency","data_value":"% of Respondents","sample_size":"Sample Size","gender":"Gender","age":"Age","race":"Race","education":"Education"})
renamed_df.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
0,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School,-106.240581,34.520881
1,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School,-97.521070,35.472031
2,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School,-75.543970,42.827001
3,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School,-86.631861,32.840571
4,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School,-81.045371,33.998821


In [175]:
Minus_Nat= renamed_df.loc[renamed_df["State"] !="National (States and DC)"]
Minus_PR = Minus_Nat.loc[Minus_Nat["State"] !="Puerto Rico"]
Minus_NatPRGuam = Minus_PR.loc[Minus_PR["State"] !="Guam"]
Minus_NatPRGuam.head()
#Minus_Ohio['C'] = np.arange(len(Minus_Ohio))
#print (Minus_Ohio)

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
0,2011,New Mexico,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,Middle School,-106.240581,34.520881
1,2005,Oklahoma,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,3.1,128,Overall,All Ages,Hispanic,High School,-97.521070,35.472031
2,2005,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.4,700,Overall,All Ages,Asian,High School,-75.543970,42.827001
3,1997,Alabama,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,Asian,High School,-86.631861,32.840571
4,2007,South Carolina,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,694,Female,All Ages,All Races,High School,-81.045371,33.998821


In [176]:
USA=Minus_NatPRGuam.dropna(how='any')
USA.head()
USA.count()

Year                22644
State               22644
Use Type            22644
Status              22644
Frequency           22644
% of Respondents    22644
Sample Size         22644
Gender              22644
Age                 22644
Race                22644
Education           22644
Lat                 22644
Lon                 22644
dtype: int64

In [177]:
National= renamed_df.loc[renamed_df["State"] =="National (States and DC)"]
National.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
2366,2015,National (States and DC),E-Cigarette Use (Youth),User Status,Current E-Cigarette User,25.8,721,Overall,All Ages,Multiple Race,High School,-95.712891,37.09024
7236,2011,National (States and DC),Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,5.4,7450,Female,All Ages,All Races,High School,-95.712891,37.09024
7244,1995,National (States and DC),Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,NaN,NaN,Overall,All Ages,Pacific Islander,High School,-95.712891,37.09024
7251,2013,National (States and DC),Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,39.6,6467,Female,All Ages,All Races,High School,-95.712891,37.09024
7261,2009,National (States and DC),Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,22.5,6698,Overall,All Ages,White,High School,-95.712891,37.09024


In [178]:
no_null_National =National.dropna(how='any')
no_null_National.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
2366,2015,National (States and DC),E-Cigarette Use (Youth),User Status,Current E-Cigarette User,25.8,721,Overall,All Ages,Multiple Race,High School,-95.712891,37.09024
7236,2011,National (States and DC),Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,5.4,7450,Female,All Ages,All Races,High School,-95.712891,37.09024
7251,2013,National (States and DC),Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,39.6,6467,Female,All Ages,All Races,High School,-95.712891,37.09024
7261,2009,National (States and DC),Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,22.5,6698,Overall,All Ages,White,High School,-95.712891,37.09024
7282,2003,National (States and DC),Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,3.9,372,Overall,All Ages,Asian,High School,-95.712891,37.09024


In [179]:
Ohio_df= renamed_df.loc[renamed_df["State"] =="Ohio"]
Ohio_df.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
149,2013,Ohio,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,NaN,NaN,Overall,All Ages,Asian,High School,-82.40426,40.06021
246,2005,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,NaN,NaN,Overall,All Ages,Multiple Race,High School,-82.40426,40.06021
287,2007,Ohio,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,NaN,NaN,Overall,All Ages,American Indian/Alaska Native,High School,-82.40426,40.06021
365,1999,Ohio,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,NaN,NaN,Overall,All Ages,Asian,High School,-82.40426,40.06021
478,1999,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,22.1,1993,Overall,All Ages,All Races,High School,-82.40426,40.06021


In [180]:
no_null_OH =Ohio_df.dropna(how='any')
no_null_OH.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
478,1999,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,22.1,1993,Overall,All Ages,All Races,High School,-82.40426,40.06021
494,2003,Ohio,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,4,544,Male,All Ages,All Races,High School,-82.40426,40.06021
572,2005,Ohio,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,22.2,661,Male,All Ages,All Races,High School,-82.40426,40.06021
696,2011,Ohio,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,51.5,1339,Overall,All Ages,All Races,High School,-82.40426,40.06021
831,2013,Ohio,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.2,864,Overall,All Ages,White,High School,-82.40426,40.06021


In [181]:
no_null_National.dtypes

Year                 object
State                object
Use Type             object
Status               object
Frequency            object
% of Respondents     object
Sample Size          object
Gender               object
Age                  object
Race                 object
Education            object
Lat                 float64
Lon                 float64
dtype: object

In [182]:
#no_null_State.t0 float
no_null_National[['% of Respondents']]=no_null_National[['% of Respondents']].astype(float)

In [183]:
no_null_OH[['% of Respondents']]=no_null_OH[['% of Respondents']].astype(float)

In [51]:
#state_counts = converted_df["Race"].value_counts()
#state_counts.head()
# Finding the average weight of all students
#no_null_State["% of Respondents"].mean()

In [184]:
no_null_National.dtypes

Year                 object
State                object
Use Type             object
Status               object
Frequency            object
% of Respondents    float64
Sample Size          object
Gender               object
Age                  object
Race                 object
Education            object
Lat                 float64
Lon                 float64
dtype: object

In [185]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_usa_df = no_null_National.groupby(['Year','Use Type','Status','Frequency','Gender','Age','Race','Education'])


# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_usa_df)

# In order to be visualized, a data function must be used...
#grouped_usa_df.count()

In [186]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_OH_df = no_null_OH.groupby(['Year','Use Type','Status','Frequency','Gender','Age','Race','Education'])


# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_OH_df)

In [187]:
grouped_OH_comparison = grouped_OH_df.mean()
grouped_OH_comparison

% of Respondents  \
Year Use Type                      Status         Frequency                       Gender  Age      Race             Education                       
1993 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School              29.0   
                                                                                  Male    All Ages All Races        High School              30.4   
                                                                                  Overall All Ages African American High School              14.2   
                                                                                                   All Races        High School              29.7   
                                                                                                   White            High School              32.8   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School              66.0   
                                                                                  Male    All Ages All Races        High School              71.0   
                                                                                  Overall All Ages African American High School              65.1   
                                                                                                   All Races        High School              68.6   
                                                                                                   White            High School              68.8   
                                                  Frequent Cigarette Smoker       Female  All Ages All Races        High School              12.3   
                                                                                  Male    All Ages All Races        High School              16.1   
                                                                                  Overall All Ages African American High School               3.3   
                                                                                                   All Races        High School              14.3   
                                                                                                   White            High School              16.6   
1997 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School              32.0   
                                                                                  Male    All Ages All Races        High School              36.9   
                                                                                  Overall All Ages African American High School              19.6   
                                                                                                   All Races        High School              34.5   
                                                                                                   White            High School              38.1   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School              67.8   
                                                                                  Male    All Ages All Races        High School              70.5   
                                                                                  Overall All Ages African American High School              67.3   
                                                                                                   All Races        High School              69.2   
                                                                                                   White            High School              69.0   
                                                  Frequent Cigarette Smoker       Female  All Ages All Rac

In [188]:
grouped_usa_comparison = grouped_usa_df.mean()
grouped_usa_comparison

% of Respondents  \
Year Use Type                      Status         Frequency                       Gender  Age      Race                          Education                       
1993 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races                     High School              31.2   
                                                                                  Male    All Ages All Races                     High School              29.8   
                                                                                  Overall All Ages African American              High School              15.4   
                                                                                                   All Races                     High School              30.5   
                                                                                                   American Indian/Alaska Native High School              35.8   
                                                                                                   Asian                         High School              20.5   
                                                                                                   Hispanic                      High School              28.7   
                                                                                                   White                         High School              33.7   
                                                  Ever Cigarette Smoker           Female  All Ages All Races                     High School              68.7   
                                                                                  Male    All Ages All Races                     High School              70.1   
                                                                                  Overall All Ages African American              High School              67.1   
                                                                                                   All Races                     High School              69.5   
                                                                                                   American Indian/Alaska Native High School              75.8   
                                                                                                   Asian                         High School              55.7   
                                                                                                   Hispanic                      High School              71.8   
                                                                                                   White                         High School              70.2   
                                                  Frequent Cigarette Smoker       Female  All Ages All Races                     High School              13.5   
                                                                                  Male    All Ages All Races                     High School              14.0   
                                                                                  Overall All Ages African American              High School               4.6   
                                                                                                   All Races                     High School              13.8   
                                                                                                   American Indian/Alaska Native High School              23.0   
                                                                                                   Asian                         High School               9.9   
                                                                                                   Hispanic                      High School               7.7   
                                                                                            

In [189]:
combined_df = pd.merge(grouped_usa_comparison, grouped_OH_comparison,on=['Year','Use Type','Status','Frequency','Gender','Age','Race','Education'])
combined_df

% of Respondents_x  \
Year Use Type                      Status         Frequency                       Gender  Age      Race             Education                         
1993 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School                31.2   
                                                                                  Male    All Ages All Races        High School                29.8   
                                                                                  Overall All Ages African American High School                15.4   
                                                                                                   All Races        High School                30.5   
                                                                                                   White            High School                33.7   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School                68.7   
                                                                                  Male    All Ages All Races        High School                70.1   
                                                                                  Overall All Ages African American High School                67.1   
                                                                                                   All Races        High School                69.5   
                                                                                                   White            High School                70.2   
                                                  Frequent Cigarette Smoker       Female  All Ages All Races        High School                13.5   
                                                                                  Male    All Ages All Races        High School                14.0   
                                                                                  Overall All Ages African American High School                 4.6   
                                                                                                   All Races        High School                13.8   
                                                                                                   White            High School                16.1   
1997 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School                34.7   
                                                                                  Male    All Ages All Races        High School                37.7   
                                                                                  Overall All Ages African American High School                22.7   
                                                                                                   All Races        High School                36.4   
                                                                                                   White            High School                39.7   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School                69.3   
                                                                                  Male    All Ages All Races        High School                70.9   
                                                                                  Overall All Ages African American High School                68.4   
                                                                                                   All Races        High School                70.2   
                                                                                                   White            High School                70.4   
                                                  Fr

In [190]:
combined_df = combined_df.rename(columns={"% of Respondents_x":"% of USA","% of Respondents_y":"% of Ohio"})

combined_df

% of USA  \
Year Use Type                      Status         Frequency                       Gender  Age      Race             Education               
1993 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School      31.2   
                                                                                  Male    All Ages All Races        High School      29.8   
                                                                                  Overall All Ages African American High School      15.4   
                                                                                                   All Races        High School      30.5   
                                                                                                   White            High School      33.7   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School      68.7   
                                                                                  Male    All Ages All Races        High School      70.1   
                                                                                  Overall All Ages African American High School      67.1   
                                                                                                   All Races        High School      69.5   
                                                                                                   White            High School      70.2   
                                                  Frequent Cigarette Smoker       Female  All Ages All Races        High School      13.5   
                                                                                  Male    All Ages All Races        High School      14.0   
                                                                                  Overall All Ages African American High School       4.6   
                                                                                                   All Races        High School      13.8   
                                                                                                   White            High School      16.1   
1997 Cigarette Use (Youth)         Smoking Status Current Cigarette Smoker        Female  All Ages All Races        High School      34.7   
                                                                                  Male    All Ages All Races        High School      37.7   
                                                                                  Overall All Ages African American High School      22.7   
                                                                                                   All Races        High School      36.4   
                                                                                                   White            High School      39.7   
                                                  Ever Cigarette Smoker           Female  All Ages All Races        High School      69.3   
                                                                                  Male    All Ages All Races        High School      70.9   
                                                                                  Overall All Ages African American High School      68.4   
                                                                                                   All Races        High School      70.2   
                                                                                                   White            High School      70.4   
                                                  Frequent Cigarette Smoker       Female  All Ages All Races        High School      15.7   
                                                                                  Male    All Ages All Races        High School      17.6   
                                        

In [191]:
Twentythree_df = USA.loc[USA["Year"] =="2003"]
Twentythree_df.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
49,2003,Georgia,Cigar Use (Youth),Smoking Status,Current Cigar Smoker,14.7,1080,Overall,All Ages,White,High School,-83.627580,32.839681
64,2003,New York,Smokeless Tobacco Use (Youth),User Status,Current Smokeless Tobacco User,6.7,4437,Male,All Ages,All Races,High School,-75.543970,42.827001
72,2003,New York,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,9.9,4257,Male,All Ages,All Races,High School,-75.543970,42.827001
102,2003,Maine,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,8.4,1646,Overall,All Ages,White,Middle School,-68.985031,45.254229
109,2003,Mississippi,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,47.1,593,Overall,All Ages,African American,Middle School,-89.538031,32.745510


In [192]:
Twentythree_df = Twentythree_df.loc[Twentythree_df["Race"] =="All Races"]
Twentythree_df = Twentythree_df.loc[Twentythree_df["Gender"] =="Overall"]


Twentythree_df

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
230,2003,Georgia,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,8.7,1934,Overall,All Ages,All Races,High School,-83.627580,32.839681
262,2003,Massachusetts,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.4,3600,Overall,All Ages,All Races,High School,-72.082691,42.276870
328,2003,West Virginia,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,28.5,1664,Overall,All Ages,All Races,High School,-80.712640,38.665510
346,2003,Vermont,Smokeless Tobacco Use (Youth),User Status,Current Smokeless Tobacco User,5.2,6019,Overall,All Ages,All Races,High School,-72.517641,43.625381
602,2003,Florida,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,18.1,3902,Overall,All Ages,All Races,High School,-81.928961,28.932040
657,2003,New York,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,1.1,9209,Overall,All Ages,All Races,High School,-75.543970,42.827001
740,2003,Florida,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,7.5,3902,Overall,All Ages,All Races,High School,-81.928961,28.932040
789,2003,Maine,Cigar Use (Youth),Smoking Status,Current Cigar Smoker,11.1,1671,Overall,All Ages,All Races,High School,-68.985031,45.254229
1043,2003,Ohio,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,55.3,1145,Overall,All Ages,All Races,High School,-82.404260,40.060210
1083,2003,Alabama,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.4,1084,Overall,All Ages,All Races,High School,-86.631861,32.840571


In [193]:
Twentythree_df = Twentythree_df.loc[Twentythree_df["State"] =="Ohio"]
Twentythree_df

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
1043,2003,Ohio,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,55.3,1145,Overall,All Ages,All Races,High School,-82.40426,40.06021
1549,2003,Ohio,Smokeless Tobacco Use (Youth),User Status,Current Smokeless Tobacco User,8,1200,Overall,All Ages,All Races,High School,-82.40426,40.06021
1585,2003,Ohio,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,22.2,1146,Overall,All Ages,All Races,High School,-82.40426,40.06021
3089,2003,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,11,1146,Overall,All Ages,All Races,High School,-82.40426,40.06021
5618,2003,Ohio,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.2,1206,Overall,All Ages,All Races,High School,-82.40426,40.06021
8616,2003,Ohio,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,2.2,1200,Overall,All Ages,All Races,High School,-82.40426,40.06021
37221,2003,Ohio,Cigar Use (Youth),Smoking Status,Current Cigar Smoker,13.6,1206,Overall,All Ages,All Races,High School,-82.40426,40.06021


In [194]:
Twentythree_df.count()

Year                7
State               7
Use Type            7
Status              7
Frequency           7
% of Respondents    7
Sample Size         7
Gender              7
Age                 7
Race                7
Education           7
Lat                 7
Lon                 7
dtype: int64

In [195]:
Twentythirteen_df = USA.loc[USA["Year"] =="2013"]
Twentythirteen_df.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
11,2013,North Dakota,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,2.2,1006,Male,All Ages,All Races,High School,-100.118421,47.475320
26,2013,Alaska,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,3.7,599,Female,All Ages,All Races,High School,-147.722059,64.845080
39,2013,Illinois,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,14.1,3036,Overall,All Ages,All Races,High School,-88.997710,40.485010
46,2013,Wisconsin,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,1.4,123,Overall,All Ages,Asian,High School,-89.816371,44.393191
60,2013,Nebraska,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,31.9,1805,Overall,All Ages,All Races,High School,-99.365721,41.641041


In [196]:
Twentythirteen_df = Twentythirteen_df.loc[Twentythirteen_df["Race"] =="All Races"]
Twentythirteen_df = Twentythirteen_df.loc[Twentythirteen_df["Gender"] =="Overall"]

Twentythirteen_df.head()

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
39,2013,Illinois,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,14.1,3036,Overall,All Ages,All Races,High School,-88.997710,40.485010
60,2013,Nebraska,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,31.9,1805,Overall,All Ages,All Races,High School,-99.365721,41.641041
66,2013,Virginia,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,0.2,8126,Overall,All Ages,All Races,Middle School,-78.457890,37.542681
98,2013,Arizona,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,14.1,1515,Overall,All Ages,All Races,High School,-111.763811,34.865970
152,2013,West Virginia,Cigarette Use (Youth),Smoking Status,Ever Cigarette Smoker,24.5,2041,Overall,All Ages,All Races,Middle School,-80.712640,38.665510


In [197]:
Twentythirteen_df.count()

Year                387
State               387
Use Type            387
Status              387
Frequency           387
% of Respondents    387
Sample Size         387
Gender              387
Age                 387
Race                387
Education           387
Lat                 387
Lon                 387
dtype: int64

In [198]:
Twentythirteen_df = Twentythirteen_df.loc[Twentythirteen_df["State"] =="Ohio"]
Twentythirteen_df

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,Lat,Lon
1850,2013,Ohio,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,3,1448,Overall,All Ages,All Races,High School,-82.40426,40.06021
6402,2013,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,6.8,1450,Overall,All Ages,All Races,High School,-82.40426,40.06021
12581,2013,Ohio,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.4,1429,Overall,All Ages,All Races,High School,-82.40426,40.06021
12737,2013,Ohio,Smokeless Tobacco Use (Youth),User Status,Current Smokeless Tobacco User,8.6,1448,Overall,All Ages,All Races,High School,-82.40426,40.06021
17471,2013,Ohio,Cigar Use (Youth),Smoking Status,Current Cigar Smoker,11.5,1429,Overall,All Ages,All Races,High School,-82.40426,40.06021
21233,2013,Ohio,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,15.1,1450,Overall,All Ages,All Races,High School,-82.40426,40.06021


In [106]:
Twentythirteen_df = Twentythirteen_df.loc[Twentythirteen_df["State"] =="Ohio"]
Twentythirteen_df

,Year,State,Use Type,Status,Frequency,% of Respondents,Sample Size,Gender,Age,Race,Education,geolocation
1850,2013,Ohio,Smokeless Tobacco Use (Youth),User Status,Frequent Smokeless Tobacco User,3,1448,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."
6402,2013,Ohio,Cigarette Use (Youth),Smoking Status,Frequent Cigarette Smoker,6.8,1450,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."
12581,2013,Ohio,Cigar Use (Youth),Smoking Status,Frequent Cigar Smoker,1.4,1429,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."
12737,2013,Ohio,Smokeless Tobacco Use (Youth),User Status,Current Smokeless Tobacco User,8.6,1448,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."
17471,2013,Ohio,Cigar Use (Youth),Smoking Status,Current Cigar Smoker,11.5,1429,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."
21233,2013,Ohio,Cigarette Use (Youth),Smoking Status,Current Cigarette Smoker,15.1,1450,Overall,All Ages,All Races,High School,"{'type': 'Point', 'coordinates': [-82.40426005..."


In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)